This notebook shows how to use `optimex` with a case study of optimizing the transition of an energy system. The case study is highly simplified, not meant to reflect the complexity of energy systems but rather to demonstrate how to use `optimex`. Its structure is based on the demonstration notebooks of [`bw_timex`](https://github.com/brightway-lca/bw_timex/tree/main/notebooks)

In [3]:
import bw2data as bd
import bw2calc as bc
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

bd.projects.set_current("optimex_ma")

## Case study setup

The `optimex` package itself does not provide any data - specifying prospective and dynamic information is up to the user. In this example, we use data from ecoinvent v3.10, and create a set of prospective databases with [premise](https://github.com/polca/premise). We applied projections for the future electricity sectors using the SSP2-RCP19 pathway from the IAM IMAGE. We selected this pathway to simply demonstrate some future development in this case study, and many other models and pathways are available. In the premise documentation you can find instructions for the creation of prospective background databases.

In [4]:
ecoinvent_vs = "3.10"

# eidb = bd.Database(f"ecoinvent-{ecoinvent_vs}-cutoff")
db_2020 = bd.Database(f"ei{ecoinvent_vs}-SSP2-RCP19-2020")
db_2030 = bd.Database(f"ei{ecoinvent_vs}-SSP2-RCP19-2030")
db_2040 = bd.Database(f"ei{ecoinvent_vs}-SSP2-RCP19-2040")
db_2050 = bd.Database(f"ei{ecoinvent_vs}-SSP2-RCP19-2050")

First, delete existing and create a new foreground database to guarentee a clean slate:

In [5]:
if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.write({})
foreground.register()

The available technologies for our transition need to be part of the foreground system. Therefore, we extract multiple processes that supply heat and/or electricity from the background system and copy them into the foreground system. This step is essential to enable the temporal disaggregation of these processes, allowing us to model their behavior and impacts over time accurately. Where available we use processes in Germany, otherwise from Europe.

In [6]:
original_bg_activities_el = {
    "wind_onshore": db_2020.get(name="electricity production, wind, 1-3MW turbine, onshore", location="DE"),
    "wind_offshore": db_2020.get(name="electricity production, wind, 1-3MW turbine, offshore", location="DE"),
    "hydro_reservoir": db_2020.get(name="electricity production, hydro, reservoir, non-alpine region", location="DE"),
    "hydro_run_of_river": db_2020.get(name="electricity production, hydro, run-of-river", location="DE"),
    "lignite": db_2020.get(name="electricity production, lignite", location="DE"),
    "photovoltaic": db_2020.get(name="electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted", location="DE"),
    "wood_chips_cogen": db_2020.get(name="heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014", location="DE", unit="kilowatt hour"),
    "bio_gas_cogen": db_2020.get(name="heat and power co-generation, biogas, gas engine", location="DE", unit="kilowatt hour"),
    "fuel_cell_cogen": db_2020.get(name="electricity, residential, by conversion of hydrogen using fuel cell, PEM, allocated by exergy, distributed by pipeline, produced by Electrolysis, PEM using electricity from grid", unit="kilowatt hour"), 
}

original_bg_activities_heat = {
    "natural_gas_boiler": db_2020.get(name="heat production, natural gas, at boiler modulating >100kW", location="WEU", unit="megajoule"),
    "wood_chips_cogen": db_2020.get(name="heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014", location="DE", unit="megajoule"),
    "bio_gas_cogen": db_2020.get(name="heat and power co-generation, biogas, gas engine", location="DE", unit="megajoule"),
    "fuel_cell_cogen": db_2020.get(name="heat, residential, by conversion of hydrogen using fuel cell, PEM, allocated by exergy, distributed by pipeline, produced by Electrolysis, PEM using electricity from grid", unit="megajoule"), 
}

for key, act in original_bg_activities_el.items():
    fg_act = act.copy(code=f"elec_prod_{key}", database=foreground.name)
    fg_act["name"] = f"electricity production, {key}, time-explicit"
    fg_act["functional flow"] = "electricity"
    fg_act.save()

for key, act in original_bg_activities_heat.items():
    fg_act = act.copy(f"heat_prod_{key}", database=foreground.name)
    fg_act["name"] = f"heat production, {key}, time-explicit"
    fg_act["functional flow"] = "heat"
    fg_act.save()


| Power Plant Type                          | Expected Lifetime (Years) |
|-------------------------------------------|---------------------------|
| Wind Turbine                              | 20–25                    |
| Lignite Conventional Power Plant          | 40–50                    |
| Natural Gas Combined Cycle Power Plant    | 25–40                    |
| CHP with Wood Chips                       | 20–30                    |
| CHP with Bio Gas                          | 20-30                    |
| Hydro                                     | 40-80                    |
| Municipal Waste Incineration Power Plant  | 25–35                    |
| Fuel Cell (PEM)                           | 5–15                     |
| Photovoltaic                              | 25–30                    |


### Assigning Temporal Information to Life Cycle Stages

To organize the timing of flows (like resources or emissions) in a process, we divide them into three life cycle stages: **construction, operation, decomission**

For every flow in a process, we assign it to one of these stages based on its context. For example:
- Materials and energy used during setup belong to the **construction** stage.
- Emissions or resource use during active operation go into the **operation** stage.
- End-of-life impacts, such as waste from dismantling, belong to the **decommissioning** stage.

We use specific distributions to spread the impacts of each flow across its corresponding stage:

- **Construction**: A **triangular distribution** is applied, which means the flow is concentrated more heavily around a central peak but still allows for earlier and later occurrences.
- **Operation**: A **uniform distribution** is used, assuming the impacts are evenly spread over the entire operational lifetime.
- **Decommissioning**: Typically, this stage occurs at the end of the process lifetime, so impacts are concentrated there.

All processes and their temporal parameters are summarized in the following table:

| Process                  | elec. | heat | lifetime [years] | construction time [years] |
|--------------------------|-------|------|------------------|---------------------------|
| Wind (on-/offshore)      | x     |      | 22               | 2                         |
| Hydro (reservoir/river)  | x     |      | 50               | 3                         |
| Lignite                  | x     |      | 45               | 3                         |
| Photovoltaic             | x     |      | 27               | 1                         |
| Natural gas boiler       |       | x    | 20               | 1                         |
| Biogas co-generation     | x     | x    | 25               | 2                         |
| Wood chips co-generation | x     | x    | 25               | 2                         |
| Fuel cell co-generation  | x     | x    | 10               | 1                         |



In [6]:
temporal_dict = {
    "wind_onshore": (22, 2),
    "wind_offshore": (22, 2),
    "hydro_reservoir": (50, 3),
    "hydro_run_of_river": (50, 3),
    "lignite": (45, 3),
    "photovoltaic": (27, 1),
    "natural_gas_boiler": (20, 1),
    "wood_chips_cogen": (25, 2),
    "bio_gas_cogen": (25, 2),
    "fuel_cell_cogen": (10, 1),
    "municipal_waste": (30, 2),
}

*Currently, we identify intermediate flows belonging in the construction phase manually and neglect the phase of decomissioning. In the future, we would like to implement this automated based on process metadata*

In [7]:
#TODO: check for potential improvements for mapping quality based on metadata
import numpy as np
import bw_temporalis as bwt

def set_temporal_distribution(activity: bd.backends.proxies.Activity, construction_time, lifetime):  
    td_construction = bwt.easy_timedelta_distribution(
        start=0,
        end=construction_time,
        resolution="Y",
        steps=(construction_time + 1),
        kind="uniform" if construction_time < 2 else "triangular",
    )

    td_use_phase = bwt.easy_timedelta_distribution(
        start=construction_time,
        end=construction_time+lifetime,
        resolution="Y",
        steps=(lifetime + 1),
        kind="uniform",
    )

    td_decommissioning = bwt.TemporalDistribution(
        date=np.array([construction_time + lifetime + 1], dtype="timedelta64[Y]"), amount=np.array([1])
    )

    # construction_keywords = ["construct", "build", "install", "erect", "assemble", "fabricate", "manufacture"]
    # operation_keywords = ["use", "operation", "maintenance", "yearly" , "annual", "supply"]
    # end_of_life_keywords = ["end of life", "decommission", "disposal", "recycling"]
    for exc in activity.exchanges():
        phase = None

        # Determine the lifecycle phase based on the exchange name
        # exc_name = exc.input["name"].lower()
        # exc_comment = exc.input.get("comment", "").lower()
        # if any(keyword in exc_name or exc_comment for keyword in construction_keywords):
        #     phase = "construction"
        # elif any(keyword in exc_name or exc_comment for keyword in end_of_life_keywords):
        #     phase = "decommissioning"
        # elif any(keyword in exc_name or exc_comment for keyword in operation_keywords):
        #     phase = "use"

        if phase == None or exc["type"] == "production": # default to operation
            phase = "use"

        hardcoded_construction_exchanges = [
            "market for dust collector, electrostatic precipitator, for industrial use",
            "market for furnace, wood chips, with silo, 5000kW",
            "market for heat and power co-generation unit, organic Rankine cycle, 1000kW electrical",
            "fuel cell system assembly, 1 kWe, proton exchange membrane (PEM)",
            "market for photovoltaic slanted-roof installation, 3kWp, multi-Si, panel, mounted, on roof",
            "lignite power plant construction",
            "market for wind power plant, 2MW, offshore, fixed parts",
            "market for wind power plant, 2MW, offshore, moving parts",
            "market for wind turbine, 2MW, onshore",
            "market for wind turbine network connection, 2MW, onshore",
            "market for transport, freight, lorry 7.5-16 metric ton, EURO3",
            "gas power plant construction, combined cycle, 400MW electrical",
            "market for hydropower plant, run-of-river",
            "market for hydropower plant, reservoir, non-alpine regions",
            "heat and power co-generation unit construction, 160kW electrical, common components for heat+electricity",
            "heat and power co-generation unit construction, 160kW electrical, components for electricity only",
            "heat and power co-generation unit construction, 160kW electrical, components for heat only",  
            "industrial furnace production, natural gas", 
        ]

        if exc["name"] in hardcoded_construction_exchanges:
            phase = "construction"
            
        if phase == "construction":
            exc["temporal_distribution"] = td_construction
        elif phase == "use":
            exc["temporal_distribution"] = td_use_phase
        elif phase == "decommissioning":
            exc["temporal_distribution"] = td_decommissioning
        exc.save()
    return activity

for act in foreground:
    key = next((k for k in temporal_dict if k in act['name'].lower()), None)
    if key:
        lifetime, construction_time = temporal_dict[key]
        set_temporal_distribution(act, construction_time, lifetime)
    act.save()

activity: heat production, wood_chips_cogen, time-explicit, exchange: market for dust collector, electrostatic precipitator, for industrial use, phase: construction
activity: heat production, wood_chips_cogen, time-explicit, exchange: market for furnace, wood chips, with silo, 5000kW, phase: construction
activity: heat production, wood_chips_cogen, time-explicit, exchange: market for heat and power co-generation unit, organic Rankine cycle, 1000kW electrical, phase: construction
activity: electricity production, lignite, time-explicit, exchange: lignite power plant construction, phase: construction
activity: electricity production, wind_onshore, time-explicit, exchange: market for transport, freight, lorry 7.5-16 metric ton, EURO3, phase: construction
activity: electricity production, wind_onshore, time-explicit, exchange: market for wind turbine network connection, 2MW, onshore, phase: construction
activity: electricity production, wind_onshore, time-explicit, exchange: market for win

The next step is to define the demand curves for electricity (measured in kWh) and heat (measured in MJ).

In [8]:
import numpy as np

demand_el = np.asarray([
    0, 0, 0, 0, 0, 56157, 56437, 55606, 57717, 57773, 55931, 60914, 71943, 84358, 
    98228, 107249, 112607, 111032, 106403, 104333, 102018, 105039, 111936, 111687, 
    110604, 113000, 114266, 119847, 111721, 115637
]) * 1e3

demand_heat = np.asarray([
    0, 0, 0, 0, 0, 262809, 269709, 273258, 298326, 344994, 382277, 428253, 434335, 
    442742, 436880, 444756, 434184, 409388, 408375, 414480, 445917, 443459, 439623, 
    419441, 429527, 423834, 414716, 414119, 407104, 407320
]) * 1e3

years = np.arange(2020, 2050)

td_demand_el = bwt.TemporalDistribution(
    date=np.arange(2020-1970, 30, dtype="datetime64[Y]"), 
    amount=demand_el
)
td_demand_heat = bwt.TemporalDistribution(
    date=np.arange(2020-1970, 30, dtype="datetime64[Y]"), 
    amount=demand_heat
)

demand_input = {
    "electricity": td_demand_el,
    "heat": td_demand_heat,
}

### Transition Pathway Optimization

We need to select an impact assessment method to minimize. Currently, `optimex` only supports the category of climate change. For the usage of dynamic characterization factors, we use `dynamic_characterization`, which needs a method as input to identify non-contributing elementary flows (cf = 0) for computational efficiency.

In [11]:
method = ('ecoinvent-3.10', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')

`optimex` also needs to know the representative timiming of the databases:

In [35]:
from datetime import datetime

optimex_type = "time_explicit"

database_date_dict = {
    db_2020.name: datetime.strptime("2020", "%Y"), 
    db_2030.name: datetime.strptime("2030", "%Y"),
    db_2040.name: datetime.strptime("2040", "%Y"),
    db_2050.name: datetime.strptime("2050", "%Y"),  
    foreground.name: "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

We also need to define a corresponding interval for time-explicit assessment, which should include the predefined demand. Here, we define a start date and a horizon length. With this we can initilize our optimex class

In [36]:
from optimex import optimex

start_date = datetime.strptime("2020", "%Y")
timehorizon = 50

opt = optimex.Optimex(
    demand = demand_input,
    start_date=start_date,
    method=method,
    database_date_dict = database_date_dict,
    timehorizon=timehorizon,
)

We construct tensors (process, flow, relative time) based on the processes in the foreground system that can produce the demanded products and the assigned temporal distributions earlier.

In [37]:
technosphere_tensor, biosphere_tensor, production_tensor = opt.construct_foreground_tensors()

In [38]:
demand_matrix = opt.parse_demand()

For each intermediate flow exchanged with the background system, we collect their Life Cycle Inventory (LCI) to construct a background inventory. This inventory quantifies the amount of each elementary flow released or consumed with each temporal background database. This process is computationally intensive. To reduce the computational burden, you can choose to include only the most influential elementary flows (e.g., top 10,000). Additionally, we provide the option to save and load existing inventory tensors.

In [39]:
load = False
if load:   
    inventory_tensor = opt.load_inventory_tensors('inventory_tensor.pkl')
    for key in list(inventory_tensor.keys()):
        if key[0] not in database_date_dict:
            del inventory_tensor[key]
    opt.load_inventory_tensors(inventory_tensor)
else:
    inventory_tensor = opt.sequential_inventory_tensor_calculation(cutoff=1e4)
    opt.save_inventory_tensors('inventory_tensor.pkl')


Since we only have temporal databases for every decade, we need to interpolate the data between the given databases. We construct a linear interpolation matrix between time and background matrices.

In [40]:
mapping_matrix = opt.construct_mapping_matrix()

For characterization, we use dynamic characterization factors with absolute global warming potential, integrating radiative forcing of elementary flow from the emission time point till the end of the time horizon. For more information, check [`dynamic_characterization`](https://github.com/brightway-lca/dynamic_characterization).

In [41]:
dynamic_characterization = (optimex_type != "static")
characterization_matrix = opt.construct_characterization_matrix(dynamic_characterization, metric="GWP")

c:\Users\HP\Desktop\Repos\optimex-1\venv\Lib\site-packages\dynamic_characterization\dynamic_characterization.py:81: UserWarning: No custom dynamic characterization functions provided. Using default dynamic characterization functions.                The flows that are characterized are based on the selection of the initially chosen impact category.                You can look up the mapping in the bw_timex.dynamic_characterizer.characterization_function_dict.
  warnings.warn(
C:\Users\HP\Desktop\Repos\optimex-1\optimex\optimex.py:569: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  df_characterized["date"] = df_characterized["date"].dt.to_pydatetime()


Co-generation processes are modeled as two different processes with a different reference flow each and emission allocation. We need to ensure that they produce heat and electricity in the correct ratio. Be advised that the factors are high because they include a unit change factor of 3.6 MJ/kWh.

In [ ]:
# additional constraint coupling capacity of heat and electricity co-generation
name_process_dict = {v: k for k, v in opt.processes.items()}
process_coupling = {
    (name_process_dict["heat production, wood_chips_cogen, time-explicit"],
        name_process_dict["electricity production, wood_chips_cogen, time-explicit"]): 10.8,
    (name_process_dict["heat production, fuel_cell_cogen, time-explicit"],
        name_process_dict["electricity production, fuel_cell_cogen, time-explicit"]): 3.6,
    (name_process_dict["heat production, bio_gas_cogen, time-explicit"],
        name_process_dict["electricity production, bio_gas_cogen, time-explicit"]): 6.18 ,
}

We use a converter in `optimex` that extracts all given parameters to combine them in a form useful for building the optimization problem. It also checks for the validity of given inputs and serves as an interface to save and load generated parameters.

In [42]:
from optimex.converter import Converter

converter = Converter(opt)
constrained_inputs = converter.combine_and_check(**{
    "process_coupling": process_coupling,
})
converter.pickle_model_inputs(f"{optimex_type}_t{timehorizon}_opt_inputs.pkl")